In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10,10)
mpl.rcParams['axes.grid'] = False
import re
import numpy as np
from PIL import Image
import time
import functools
import os
import pandas as pd
import librosa
import glob
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import librosa.display
import hashlib
from os import listdir
from os.path import isfile, join
from os import walk

import glob
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
from keras.models import load_model


Using TensorFlow backend.


In [2]:
X_np_training = np.load('X_training.npy')
Y_np_training = np.load('Y_training.npy') 
X_np_validation = np.load('X_validation.npy') 
Y_np_validation = np.load('Y_validation.npy')
X_np_testing = np.load('X_testing.npy')
Y_np_testing = np.load('Y_testing.npy')
Y_oh_testing = np.load('Y_oh_testing.npy')
Y_oh_validation = np.load('Y_oh_validation.npy')
Y_oh_training = np.load('Y_oh_training.npy') 

In [15]:
from tensorflow_model_optimization.sparsity import keras as sparsity
num_train_samples = X_np_training.shape[0]
batch_size = 32

In [16]:
loaded_model = tf.keras.models.load_model('my_model1.h5')

epochs = 4
end_step = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * epochs
print(end_step)

new_pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                   final_sparsity=0.90,
                                                   begin_step=0,
                                                   end_step=end_step,
                                                   frequency=100)
}

new_pruned_model = sparsity.prune_low_magnitude(loaded_model, **new_pruning_params)
new_pruned_model.summary()

new_pruned_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])

10608


W0812 11:06:30.636957 19916 deprecation.py:323] From C:\Users\karth\Anaconda3\lib\site-packages\tensorflow_model_optimization\python\core\sparsity\keras\pruning_schedule.py:240: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_lstm_3 ( (None, 32)                19587     
_________________________________________________________________
prune_low_magnitude_activati (None, 32)                1         
_________________________________________________________________
prune_low_magnitude_dense_3  (None, 6)                 392       
_________________________________________________________________
prune_low_magnitude_activati (None, 6)                 1         
Total params: 19,981
Trainable params: 10,054
Non-trainable params: 9,927
_________________________________________________________________


In [21]:
callbacks = [
    sparsity.UpdatePruningStep(),
]

new_pruned_model.fit(X_np_training, Y_oh_training,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=callbacks,
          validation_data=(X_np_validation, Y_oh_validation))


Train on 84843 samples, validate on 9981 samples
Epoch 1/4
84843/84843 [==============================] - 55s 649us/sample - loss: 0.4727 - acc: 0.9056 - val_loss: 0.4569 - val_acc: 0.9073
Epoch 2/4
84843/84843 [==============================] - 54s 636us/sample - loss: 0.4595 - acc: 0.9056 - val_loss: 0.4558 - val_acc: 0.9073
Epoch 3/4
84843/84843 [==============================] - 54s 641us/sample - loss: 0.4561 - acc: 0.9056 - val_loss: 0.4526 - val_acc: 0.9073
Epoch 4/4
84843/84843 [==============================] - 54s 638us/sample - loss: 0.4527 - acc: 0.9056 - val_loss: 0.4501 - val_acc: 0.9073


NameError: name 'x_test' is not defined

In [23]:
final_model = sparsity.strip_pruning(new_pruned_model)
final_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 32)                9856      
_________________________________________________________________
activation_5 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 198       
_________________________________________________________________
activation_6 (Activation)    (None, 6)                 0         
Total params: 10,054
Trainable params: 10,054
Non-trainable params: 0
_________________________________________________________________


In [24]:
final_model.save('final_model.h5')

In [25]:
for i, w in enumerate(final_model.get_weights()):
    print(
        "{} -- Total:{}, Zeros: {:.2f}%".format(
            final_model.weights[i].name, w.size, np.sum(w == 0) / w.size * 100
        )
    )

lstm_3_4/kernel:0 -- Total:5632, Zeros: 90.00%
lstm_3_4/recurrent_kernel:0 -- Total:4096, Zeros: 89.99%
lstm_3_4/bias:0 -- Total:128, Zeros: 0.00%
dense_3_4/kernel:0 -- Total:192, Zeros: 90.10%
dense_3_4/bias:0 -- Total:6, Zeros: 0.00%


In [26]:
import tempfile
import zipfile

_, new_pruned_keras_file = tempfile.mkstemp(".h5")
print("Saving pruned model to: ", new_pruned_keras_file)
tf.keras.models.save_model(final_model, new_pruned_keras_file, include_optimizer=False)

# Zip the .h5 model file
_, zip3 = tempfile.mkstemp(".zip")
with zipfile.ZipFile(zip3, "w", compression=zipfile.ZIP_DEFLATED) as f:
    f.write(new_pruned_keras_file)
print(
    "Size of the pruned model before compression: %.2f Mb"
    % (os.path.getsize(new_pruned_keras_file) / float(2 ** 20))
)
print(
    "Size of the pruned model after compression: %.2f Mb"
    % (os.path.getsize(zip3) / float(2 ** 20))
)

Saving pruned model to:  C:\Users\karth\AppData\Local\Temp\tmp3njvdgbt.h5
Size of the pruned model before compression: 0.05 Mb
Size of the pruned model after compression: 0.01 Mb
